# VQE in DFT with PsiEmbed and Qiskit

Here we define the inputs as required by PsiEmbed. Note that we'll follow the logic of `embedding_module/run_open_shell`.

We can think of this procedure are requiring three steps:

1. Pre-embedding

    Here we define the problem, and run a low-level calculation of the whole system. From this we obtain the pre-embedded density matrices $\gamma^A$ and $\gamma^B$

    We then define the level-shift projector $P$ and embedding potential $V_{emb}$.

2. Embedding

    Using $V_{emb}$ we run a high-level method simulation of the active region to get the embedded density matrix $\gamma^A_{emb}$.

    We calculate the correction term $tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

3. Post-embedding

    Finally we calculate the embedded energy, by removing $V_{emb}$ from the Hamiltonian, and using density matrix $\gamma^A_{emb}$.

    The total energy is then given by: $E = E[\gamma^A_{emb}] + E[\gamma^B] + g[\gamma^A, \gamma^B] + E_{nuclear} + tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

# 0. Set Parameters

First we'll set the parameters 

In [1]:
from copy import copy
from typing import Dict
from spade import fill_defaults
import numpy as np
from scipy import linalg
from spade.main import driver
from spade.embedding_module import run_closed_shell

ethane = """
C      0.7516     -0.0225     -0.0209
C     -0.7516      0.0225      0.0209
H      1.1851     -0.0039      0.9875
H      1.1669      0.8330     -0.5693
H      1.1155     -0.9329     -0.5145
H     -1.1669     -0.8334      0.5687  
H     -1.1157      0.9326      0.5151
H     -1.1850      0.0044     -0.9875
    """

ethylene = """
H    2.933  -0.150  -9.521
H    2.837   1.682  -9.258
C    3.402   0.773  -9.252
C    4.697   0.791  -8.909
H    5.262  -0.118  -8.904
H    5.167   1.714  -8.641
    """

methanol = """
O     -0.6582     -0.0067      0.1730 
H     -1.1326     -0.0311     -0.6482
C      0.7031      0.0083     -0.1305
H      1.2001      0.0363      0.8431
H      0.9877      0.8943     -0.7114
H      1.0155     -0.8918     -0.6742
  """
# H     -1.1326     -0.0311     -0.6482 <--- this goes with the oxygen


formaldehyde = """
C      0.5979      0.0151      0.0688
H      1.0686     -0.1411      1.0408
H      1.2687      0.2002     -0.7717
O     -0.5960     -0.0151     -0.0686
  """

water = """
O          0.00000        0.00000        0.1653507
H          0.00000        0.7493682     -0.4424329
H          0.00000       -0.7493682     -0.4424329
    """

h_peroxide = """
O          0.00000        0.00000        0.00000
O          1.00000        0.00000        0.00000
H          0.00000        0.50000        0.00000
H          1.00000       -0.50000        0.00000
"""

LiH = """
Li 0.0 0.0 0.0
H  0.0 0.0 1.541
"""

options = {}
options['geometry'] = LiH
options['n_active_atoms'] = 1 # number of active atoms (first n atoms in the geometry string)

run_fci = True

options['basis'] = 'STO-6G' # basis set 
options['low_level'] = 'b3lyp' # level of theory of the environment 
options['high_level'] = 'mp2' # level of theory of the embedded system
options['low_level_reference'] = 'rhf'
options['high_level_reference'] = 'rhf'
options['package'] = 'pyscf'

keywords = fill_defaults(options)

#run_closed_shell(keywords)

/home/mwilliams/anaconda3/envs/mres/lib//python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


# 1. Low-level whole system calculation

The first step is to run a mean field caluclation of the whole system.

The Embed class and its subclasses have a method to do this which also sets the following properties:
    Exchange correlation potentials (v_xc_total if embedding potential is not set, or alpha/beta_v_xc_total)
    

In [2]:
from pyscf import gto, scf, cc, ao2mo, fci

mol = gto.Mole(atom=keywords['geometry'], basis=keywords['basis'], charge=0).build()

ks = scf.RKS(mol)
ks.conv_tol = keywords["e_convergence"]
ks.xc = keywords["low_level"]
e_initial = ks.kernel()

# Store the initial value of h core as this is needed later,
# but is overwritten

initial_h_core = ks.get_hcore()

mol_copy = copy(mol)

converged SCF energy = -8.04003110337844


In [3]:
ks.mo_coeff.shape

(6, 6)

In [4]:
hf = mol_copy.RHF().run()
if run_fci:
    ref_fci = fci.FCI(hf)
    ref_fci.conv_tol = keywords["e_convergence"]
    fci_result = ref_fci.kernel()

    # This DOES have nuclear energy included!
    expected_energy = fci_result[0]

else:
    ref_cc = cc.CCSD(hf)
    ref_cc.conv_tol = keywords["e_convergence"]
    cc_result = ref_cc.kernel()

    expected_energy = hf.energy_tot() + cc_result[0]

f"{expected_energy=}"


converged SCF energy = -7.95323159583687


'expected_energy=-7.972781429567789'

# 2. Orbital Localisation
Find the orbitals of the active space and environment, using SPADE.

In [5]:
n_occupied_mos = np.count_nonzero(ks.mo_occ == 2)
occupied_orbitals = ks.mo_coeff[:, :n_occupied_mos]

n_act_aos = mol.aoslice_by_atom()[keywords['n_active_atoms']-1][-1]
ao_overlap = ks.get_ovlp()

# Orbital rotation and partition into subsystems A and B
#rotation_matrix, sigma = embed.orbital_rotation(occupied_orbitals,
#    n_act_aos, ao_overlap)

rotated_orbitals = linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals
_, sigma, right_vectors = linalg.svd(rotated_orbitals[:n_act_aos, :])

#n_act_mos, n_env_mos = embed.orbital_partition(sigma)
value_diffs = sigma[:-1]-sigma[1:]
n_act_mos = np.argmax(value_diffs) + 1
n_env_mos = n_occupied_mos - n_act_mos

# Defining active and environment orbitals and 
act_orbitals = occupied_orbitals @ right_vectors.T[:, :n_act_mos]
env_orbitals = occupied_orbitals @ right_vectors.T[:, n_act_mos:]
act_density = 2.0 * act_orbitals @ act_orbitals.T
env_density = 2.0 * env_orbitals @ env_orbitals.T

In [6]:
ks.mo_occ.shape

(6,)

In [7]:
occupied_orbitals.shape

rotated_orbitals.shape

rotated_orbitals[:n_act_aos, :].shape

mol.aoslice_by_atom()

array([[0, 3, 0, 5],
       [3, 4, 5, 6]])

# 3. Calculate the cross subsytem terms

In [8]:
# Retrieving the subsytem energy terms and potential matrices

# The function called looks like this
def closed_shell_subsystem(scf, density):
        #It seems that PySCF lumps J and K in the J array 
        j = ks.get_j(dm = density)
        k = np.zeros(np.shape(j))
        two_e_term =  ks.get_veff(ks.mol, density)
        e_xc = two_e_term.exc
        v_xc = two_e_term - j

        # Energy
        e = np.einsum("ij,ij", density, ks.get_hcore() + j/2) + e_xc
        return e, e_xc, j, k, v_xc

e_act, e_xc_act, j_act, k_act, v_xc_act = (
    closed_shell_subsystem(scf, act_density))
e_env, e_xc_env, j_env, k_env, v_xc_env = (
    closed_shell_subsystem(scf, env_density))

# Computing cross subsystem terms
# Note that the matrix dot product is equivalent to the trace.
j_cross = 0.5 * (np.einsum("ij,ij",act_density, j_env)
        + np.einsum("ij,ij", env_density, j_act))

k_cross = 0.0

xc_cross = ks.get_veff().exc - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross
print(f"{e_act=},{e_xc_act=}")#, {j_act=}, {k_act=}, {v_xc_act=}")
f"{two_e_cross=}, {xc_cross=}"

e_act=-7.93328627815767,e_xc_act=-1.7095312650804746


'two_e_cross=1.445023453967674, xc_cross=-0.022400700772928883'

# 4. Define $V_{emb}$

We can now define the projector used to orthogonalise the Molecular and Atomic orbitals. From this we calculate the embedding potential.

$P_{\alpha, \beta} = S\gamma^BS$

From this we can now also define the embedding potential.

$V_{emb} = g[\gamma^A, \gamma^B] - g[\gamma^A] + \mu P$

In [9]:
# Define the mu-projector
print(f"{keywords['level_shift']=}")
projector = keywords['level_shift'] * (ks.get_ovlp() @ env_density
    @ ks.get_ovlp())

v_xc_total = ks.get_veff() - ks.get_j()

# Defining the embedded core Hamiltonian
v_emb = (j_env + v_xc_total - v_xc_act + projector)

keywords['level_shift']=1000000.0


# 5A Run HF of full system with $V_{emb}$ to get $\gamma^A_{emb}$

Here, PsiEmbed gives us the option to stop, outputting values for calculation by other means.

To continue, we run the mean field method, but with the embedding potentials as calulated.

In [10]:
embedded_scf = scf.RHF(mol_copy)
embedded_scf.conv_tol = keywords["e_convergence"]
embedded_scf.mol.nelectron = 2*n_act_mos

h_core = ks.get_hcore()

embedded_scf.get_hcore = lambda *args, **kwargs: h_core + v_emb

embedded_scf.kernel()

embedded_occ_orbs = embedded_scf.mo_coeff[:, embedded_scf.mo_occ>0]
embedded_density = 2*embedded_occ_orbs @ embedded_occ_orbs.T

e_emb = embedded_scf.energy_elec(dm=embedded_density, vhf=embedded_scf.get_veff())[0]

print(f"{e_emb=}")

converged SCF energy = -5.41262578421813
e_emb=-6.442821522543896


Overwritten attributes  get_hcore  of <class 'pyscf.scf.hf.RHF'>
/home/mwilliams/anaconda3/envs/mres/lib//python3.8/site-packages/pyscf/gto/mole.py:1089: UserWarning: Function mol.dumps drops attribute _nelectron because it is not JSON-serializable
  warnings.warn(msg)


# 6 Calculate correction term

In [11]:
# Compute the correction
# - There are two versions used for different embeddings
dm_correction = np.einsum("ij,ij", v_emb, embedded_density - act_density)
wf_correction = np.einsum("ij,ij", act_density, v_emb)

print(f"{wf_correction=}, {dm_correction=}")

wf_correction=1.4553626589970463, dm_correction=0.0007018125190256796


# 7 Calculate $E[\gamma^A_{emb}]$

We calculate the Hartree-fock energy of the embedded region, we then add correlation later.

In [12]:
# n_active_atoms = keywords["n_active_atoms"]
# part_geometry = keywords["geometry"].split("\n")[1:n_active_atoms+1][0]
# part_mol = gto.Mole(atom=part_geometry, basis=keywords["basis"])
# part_mol.nelectron == 2*n_act_mos

# part_scf = scf.RHF(part_mol)
# part_scf.kernel()

# Openfermion - Get Qubit Hamiltonian

Much of this is taken from the get_qubit_hamiltonian function of the MolecularData class in OpenFermion. However, we want to us our own PySCF molecule.

First we'll get the active and occupied indices - this will be how we reduce the number of qubits needed.

In [13]:
from pyscf import ao2mo
from openfermion.ops.representations import InteractionOperator, get_active_space_integrals
from openfermion.linalg import eigenspectrum, expectation
from openfermion.transforms import jordan_wigner

active_indices = [i for i in range(embedded_scf.mol.nao - n_env_mos)]
occupied_indices = embedded_scf.mo_occ == 2
occupied_indices = [i for i, val in enumerate(occupied_indices) if val]

print(f"{active_indices=} \n {occupied_indices=}")


active_indices=[0, 1, 2, 3, 4] 
 occupied_indices=[0]


Now we'll calculate the one and two body integrals to create a fermionic Hamiltonian.

We then cut out the contribution from the environment orbitals. (Note the reduction in the shape of the integrals!)

In [14]:
base_mol = gto.Mole(atom=keywords["geometry"], basis=keywords["basis"], charge=0).build()
base_scf = scf.RHF(base_mol)
base_scf.conv_tol = keywords["e_convergence"]
base_scf.kernel()

converged SCF energy = -7.95323159550841


-7.953231595508413

In [15]:
from functools import reduce

temp_scf = embedded_scf
temp_mol = mol
coeff = temp_scf.mo_coeff[:, active_indices]


n_orbs = len(active_indices)

one_body_integrals = coeff.T @ temp_scf.get_hcore() @ coeff

#temp_scf.get_hcore = lambda *args, **kwargs : initial_h_core
temp_mol.incore_anyway == True

# Get two electron integrals in compressed format.
two_body_compressed = ao2mo.kernel(temp_mol,
    coeff)

two_body_integrals = ao2mo.restore(1, # no permutation symmetry
    two_body_compressed, n_orbs)

# Openfermion uses pysicist notation whereas pyscf uses chemists
two_body_integrals = np.asarray(
    two_body_integrals.transpose(0, 2, 3, 1), order='C')

# # Get the contribution from active parts
# const, act_one_body, act_two_body = get_active_space_integrals(one_body_integrals, 
#     two_body_integrals,
#     active_indices=active_indices,
#     occupied_indices=occupied_indices,
#     )

print(f"{one_body_integrals.shape=}, {two_body_integrals.shape=}")
#print(f"{act_one_body.shape=}, {act_two_body.shape=}, {const=}")


one_body_integrals.shape=(5, 5), two_body_integrals.shape=(5, 5, 5, 5)
act_one_body.shape=(5, 5), act_two_body.shape=(5, 5, 5, 5), const=-6.442821522543321


We have to transform this to spin-orbital coefficients to make a qubit operator in terms of the Pauli basis.

In [25]:
from openfermion.chem.molecular_data import spinorb_from_spatial

one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_integrals, two_body_integrals)

molecular_hamiltonian = InteractionOperator(0,
                                            one_body_coefficients,
                                            0.5 * two_body_coefficients,)

Qubit_Hamiltonian = jordan_wigner(molecular_hamiltonian)


e_vqe_act = eigenspectrum(Qubit_Hamiltonian)[0]
print(f"{e_vqe_act=}")

e_nuc = base_mol.energy_nuc()

print(f"{n_act_mos=}, {n_env_mos=}")

print(e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction)
print(e_vqe_act + e_env + two_e_cross + e_nuc + dm_correction)

e_vqe_act=-6.44295997672804
n_act_mos=1, n_env_mos=1
-8.00506746094585
-6.549002989429779


In [19]:
Qubit_Hamiltonian

-2.458227168820679 [] +
-0.005576996775047178 [X0 X1 Y2 Y3] +
0.00062359109632963 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Y9] +
0.00062359109632963 [X0 X1 X3 Z4 Z5 Z6 Z7 X8] +
-0.0023590205469307623 [X0 X1 Y4 Y5] +
-0.0023590205469307623 [X0 X1 Y6 Y7] +
-0.002251267868733035 [X0 X1 Y8 Y9] +
0.005576996775047178 [X0 Y1 Y2 X3] +
-0.00062359109632963 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 X9] +
0.00062359109632963 [X0 Y1 Y3 Z4 Z5 Z6 Z7 X8] +
0.0023590205469307623 [X0 Y1 Y4 X5] +
0.0023590205469307623 [X0 Y1 Y6 X7] +
0.002251267868733035 [X0 Y1 Y8 X9] +
0.02009223212957343 [X0 Z1 X2] +
0.0010954361465115494 [X0 Z1 X2 X3 Z4 Z5 Z6 Z7 Z8 X9] +
0.0010954361465115494 [X0 Z1 X2 Y3 Z4 Z5 Z6 Z7 Z8 Y9] +
-0.00023863605984238126 [X0 Z1 X2 Z3] +
0.0035610997556535847 [X0 Z1 X2 Z4] +
0.0012206911636313262 [X0 Z1 X2 Z5] +
0.0035610997556535856 [X0 Z1 X2 Z6] +
0.001220691163631327 [X0 Z1 X2 Z7] +
0.003761292412951553 [X0 Z1 X2 Z8] +
0.002169911280732247 [X0 Z1 X2 Z9] +
-0.002340408592022259 [X0 Z1 Z2 X3 Y4 Y5] +
-0.0023404085

# Add up parts without running VQE

In [24]:
e_nuc = mol.energy_nuc()

e_mf_emb = e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction 
print("Component contributions")
print(f"{e_vqe_act=}, {e_env=}, {two_e_cross=}, {wf_correction=}, {dm_correction=}, {e_nuc=}\n")

# Print out the final value.
print(f"FCI Energy:\t\t{expected_energy}")
print(f"DFT Energy:\t\t{e_initial}")
print(f"Error:\t\t\t{(expected_energy-e_initial)*100/expected_energy:.2f}%")
print(f"Embedding Energy:\t{e_mf_emb}")
print(f"Error:\t\t\t{(expected_energy-e_mf_emb)*100/expected_energy:.2f}%")

Component contributions
e_vqe_act=-4.410584067032425, e_env=-2.581964017514202, two_e_cross=1.445023453967674, wf_correction=1.4553626589970463, dm_correction=0.0007018125190256796, e_nuc=1.0301957383257627

FCI Energy:		-7.972781429567789
DFT Energy:		-8.040031103378437
Error:			-0.84%
Embedding Energy:	-5.972691551250236
Error:			25.09%


# Pennylane VQE

Taking the qubit operator we defined with open fermion, we can perform VQE using the CCSD ansatz.

In [20]:
import pennylane as qml
from pennylane import qchem
from pennylane.templates import UCCSD
from functools import partial

h = qchem.convert_observable(Qubit_Hamiltonian)

qubits = len(h.wires)

electrons = n_act_mos*2

ref_state = qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qchem.excitations_to_wires(singles, doubles)


2

In [42]:
import qiskit
qiskit.IBMQ.save_account("4c553edd3cfbbbbba55d464b2d2a24cb6dab41a62b47a6d76e3dc6d82d4ede6338f71c8f7350868a5a9df250e908fb4ff61e5d0bfe09ca3203b264e70ec6e3e9")
qiskit.IBMQ.load_account()

# Define the device
dev = qml.device('default.qubit', wires=qubits)
#dev = qml.device('qiskit.ibmq', wires=qubits)


# Define the UCCSD ansatz
ansatz = partial(UCCSD, init_state=ref_state, s_wires=s_wires, d_wires=d_wires)

# Define the cost function
cost_fn = qml.ExpvalCost(ansatz, h, dev)

opt = qml.GradientDescentOptimizer(stepsize=0.4)

# Compute the expectation value of 'h' for given set of parameters 'params'
params = np.random.normal(0, np.pi, len(singles) + len(doubles))
#params = [-5.93771990e+00, -4.76699963e+00,  3.14131922e+00,  3.98850044e-01,
#  1.01543437e+00, -3.13990630e+00, -2.01576288e+00, -1.45564534e+00,
#  4.48723854e+00, -2.21525486e-01, -1.45345259e+00, -6.13076325e-01,
#  1.00582995e-01,  2.76812554e+00, -1.91234611e-03,]


configrc.store_credentials:WARNING:1970-01-02 02:02:56,825: Credentials already present. Set overwrite=True to overwrite.
/home/mwilliams/anaconda3/envs/mres/lib//python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
ibmqfactory.load_account:WARNING:1970-01-02 02:02:57,242: Credentials are already in use. The existing account in the session will be replaced.
/home/mwilliams/anaconda3/envs/mres/lib//python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/home/mwilliams/anaconda3/envs/mres/lib//python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/home/mwilliams/anaconda3/envs/mres/lib//python3.8/site-packages/

In [43]:
max_iterations = 100
conv_tol = keywords["e_convergence"] * 10
energy = [cost_fn(params)]
param_vals = [params]

print("Beginning Iterations")
for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost_fn, params)

    energy.append(cost_fn(params))
    param_vals.append(params)

    conv = np.abs(energy[-1] - prev_energy)

    # if n % 2 == 0:
    print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {param_vals[-1]}")

Beginning Iterations
Step = 0,  Energy = -9.18212239 Ha
Step = 1,  Energy = -9.41436480 Ha
Step = 2,  Energy = -9.63557920 Ha
Step = 3,  Energy = -9.78512646 Ha
Step = 4,  Energy = -9.87485802 Ha
Step = 5,  Energy = -9.95840932 Ha
Step = 6,  Energy = -10.09386366 Ha
Step = 7,  Energy = -10.34388238 Ha
Step = 8,  Energy = -10.73295274 Ha
Step = 9,  Energy = -11.13310345 Ha
Step = 10,  Energy = -11.35549596 Ha
Step = 11,  Energy = -11.42433925 Ha
Step = 12,  Energy = -11.44087078 Ha
Step = 13,  Energy = -11.44537783 Ha
Step = 14,  Energy = -11.44723334 Ha
Step = 15,  Energy = -11.44838926 Ha
Step = 16,  Energy = -11.44926963 Ha
Step = 17,  Energy = -11.44998401 Ha
Step = 18,  Energy = -11.45057351 Ha
Step = 19,  Energy = -11.45106190 Ha
Step = 20,  Energy = -11.45146684 Ha
Step = 21,  Energy = -11.45180256 Ha
Step = 22,  Energy = -11.45208083 Ha
Step = 23,  Energy = -11.45231142 Ha
Step = 24,  Energy = -11.45250247 Ha
Step = 25,  Energy = -11.45266073 Ha
Step = 26,  Energy = -11.45279181

In [2]:
energy

NameError: name 'energy' is not defined

# 8. Add all the parts up.

e_act_emb : $\epsilon[\gamma^A_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^B]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [25]:
e_nuc = mol.energy_nuc()

e_mf_emb = e_vqe_act + e_env + two_e_cross + e_nuc + dm_correction
print("Component contributions")
print(f"{e_vqe_act=}, {e_env=}, {two_e_cross=}, {wf_correction=}, {dm_correction=}, {e_nuc=}\n")

# Print out the final value.
print(f"FCI Energy:\t\t{expected_energy}")
print(f"DFT Energy:\t\t{e_initial}")
print(f"Error:\t\t\t{(expected_energy-e_initial)*100/expected_energy:.2f}%")
print(f"Embedding Energy:\t{e_mf_emb}")
print(f"Error:\t\t\t{(expected_energy-e_mf_emb)*100/expected_energy:.2f}%")


Component contributions
e_vqe_act=-97.82282083977448, e_env=-23.237303494494267, two_e_cross=16.18037188035646, wf_correction=16.308795641874894, dm_correction=-0.0008673600709698803, e_nuc=9.128283728282938

FCI Energy:		-75.73152218912338
DFT Energy:		-76.00109433740884
Error:			-0.36%
Embedding Energy:	-95.75233608570032
Error:			-26.44%


In [228]:
import pennylane as qml

lines = np.array([l.split() for l in keywords["geometry"].split("\n")[:-1] if l])

In [229]:
atoms = lines[:, :1]
atoms = atoms.reshape(len(lines))
coords = lines[:, 1:]
coords = coords.reshape(coords.shape[0]**2).astype(float)

In [233]:
coords[0:3]

array([0.       , 0.       , 0.1653507])

In [234]:
filepath = qml.qchem.meanfield(atoms[0], coords[0:3], basis=keywords["basis"], package="pyscf")
q_ham = qml.qchem.decompose(filepath, mapping="jordan_wigner")
print(q_ham)

(-47.41882048824246+0j) [] +
(-0.019665257071075897+0j) [X0 X1 Y2 Y3] +
(-0.006641166911403775+0j) [X0 X1 Y4 Y5] +
(-0.006641166911403775+0j) [X0 X1 Y6 Y7] +
(-0.006641166911403775+0j) [X0 X1 Y8 Y9] +
(0.019665257071075897+0j) [X0 Y1 Y2 X3] +
(0.006641166911403775+0j) [X0 Y1 Y4 X5] +
(0.006641166911403775+0j) [X0 Y1 Y6 X7] +
(0.006641166911403775+0j) [X0 Y1 Y8 X9] +
(0.14185284805199+0j) [X0 Z1 X2] +
(0.005718593068437519+0j) [X0 Z1 X2 Z3] +
(0.012174620045398191+0j) [X0 Z1 X2 Z4] +
(0.003225366008534387+0j) [X0 Z1 X2 Z5] +
(0.012174620045398191+0j) [X0 Z1 X2 Z6] +
(0.003225366008534387+0j) [X0 Z1 X2 Z7] +
(0.012174620045398191+0j) [X0 Z1 X2 Z8] +
(0.003225366008534387+0j) [X0 Z1 X2 Z9] +
(-0.008949254036863804+0j) [X0 Z1 Z2 X3 Y4 Y5] +
(-0.008949254036863804+0j) [X0 Z1 Z2 X3 Y6 Y7] +
(-0.008949254036863804+0j) [X0 Z1 Z2 X3 Y8 Y9] +
(0.008949254036863804+0j) [X0 Z1 Z2 Y3 Y4 X5] +
(0.008949254036863804+0j) [X0 Z1 Z2 Y3 Y6 X7] +
(0.008949254036863804+0j) [X0 Z1 Z2 Y3 Y8 X9] +
(0.12073426

In [136]:
energy

[-44.55861490611497,
 -35.048457428749465,
 -21.39530587659857,
 -44.6538071069298,
 -39.22216975038845,
 -39.28259296737336,
 -27.690896057044363,
 -21.460271043334075,
 -44.027963935563946,
 -43.702724400618756,
 -45.3677521505328,
 -45.358185311393505,
 -45.16723361646617,
 -42.11692787879853,
 -21.9749890164566,
 -42.21180702748919,
 -30.57208536249821,
 -43.810855828081245,
 -43.79164962214616,
 -45.023346072866794,
 -39.75857511557343,
 -19.751745898923883,
 -29.909139469761293,
 -41.39931028236084,
 -34.51012228215426,
 -22.998287099675565,
 -45.2853768767361,
 -44.337206686204404,
 -36.40444873935267,
 -44.05391909093208,
 -35.60559699897695,
 -36.75632421423436,
 -21.146436730708754,
 -45.37154749624263,
 -45.28084951629086,
 -43.773634333779285,
 -27.641846194410213,
 -29.808859547366758,
 -37.60093374372787,
 -32.298544130402405,
 -35.85662808028473,
 -31.797715590044106,
 -21.054753223426697,
 -36.68792302382612,
 -45.11079310146467,
 -43.18704815087737,
 -37.26979290525492

In [ ]:
# This is just here to quit any "Run All" commands
import pdb;pdb.set_trace()

# Tequila

In [ ]:
import tequila as tq
from tequila.quantumchemistry import ParametersQC, QuantumChemistryBase, QuantumChemistryPySCF
from tequila import ExpectationValue
from tequila.quantumchemistry.qc_base import ClosedShellAmplitudes
#tq_mol = QuantumChemistryBase.do_make_molecule(self, nuclear_repulsion=0, one_body_integrals=act_one_body, two_body_integrals=act_two_body)

In [ ]:
params = ParametersQC(basis_set=keywords["basis"], geometry=keywords["geometry"])
driver = QuantumChemistryPySCF(parameters=params, transformation="jordan_wigner", one_body_integrals=act_one_body, two_body_integrals=act_two_body)
#tq_mol = driver.make_molecule(one_body_integrals=act_one_body, two_body_integrals=act_two_body)

In [ ]:
amps = ClosedShellAmplitudes(tIA=act_one_body, tIjAb=act_two_body)
U = driver.make_uccsd_ansatz(trotter_steps=1, initial_amplitudes=amps)
H = driver.make_hamiltonian()

E = ExpectationValue(H=H, U=U)

In [ ]:
#E = tq.ExpectationValue(H=H, U=U, transformation="jordan_wigner")
result = tq.minimize(objective=E, method="cobyla", initial_values=None)

5

In [ ]:
commuting_obs = [qml.PauliX(0), qml.PauliX(0) @ qml.PauliZ(1)]
H = qml.vqe.Hamiltonian([1, 1], commuting_obs)

dev = qml.device("default.qubit", wires=2)
ansatz = qml.templates.StronglyEntanglingLayers

cost_opt = qml.ExpvalCost(ansatz, H, dev, optimize=True)
cost_no_opt = qml.ExpvalCost(ansatz, H, dev, optimize=False)

params = qml.init.strong_ent_layers_uniform(3, 2)

In [ ]:
# Can either convert to pennylane or generate a Fermionic operator in pl
import pennylane_qchem as pl
import pennylane as qml

pl_qham = pl.qchem.convert_observable(Qubit_Hamiltonian)

dev = qml.device("default.qubit", wires=n_qubits)
n_electrons = n_act_mos
hf = pl.qchem.hf_state(n_electrons, n_qubits)
print(hf)

In [ ]:
from openfermion.circuits import uccsd_generator
import cirq


In [ ]:
anzats = uccsd_generator(act_one_body, act_two_body)

circuit = cirq.Circuit()
simulator = cirq.Simulator()





## Qiskit Stuff Below here

In [22]:
qh_terms = Qubit_Hamiltonian.terms
n_qubits = molecular_hamiltonian._n_qubits

intermediate = {}
for term, value in qh_terms.items():
    # Assume I for each qubit unless explicity stated
    op_string = ["I"]*n_qubits
    for pauli in term:
        position = pauli[0]
        operator = pauli[1]

        op_string[position] = operator

    intermediate["".join(op_string)] = value

print(intermediate)

{'IIIII': -27.292796792001084, 'ZIIII': 14.089672416569032, 'IZIII': 3.3890845568584465, 'IIZII': 2.79082116168039, 'IIIZI': 3.38901526963488, 'IIIIZ': 3.634203387258335, 'XXIII': 0.07177386427593799, 'YYIII': 0.07177386427593799, 'XZZXI': 0.021981207571056316, 'YZZYI': 0.021981207571056316, 'IXZXI': -0.22601413274862364, 'IYZYI': -0.22601413274862364}


In [30]:
# Pennylane
import pennylane as qml
from pennylane import PauliX, PauliY, PauliZ, Identity

Identity(1)

opdict = {"I": Identity, "X": PauliX, "Y": PauliY, "Z": PauliZ}

# Initialise the operator with the identity contribution
values = [v for v in intermediate.values()]
qiskit_op = intermediate["I"*n_qubits] * Identity(n_qubits)
operators = [Identity(n_qubits)]

for op, value in intermediate.items():

    if op == "I"*n_qubits:
        continue

    paulis = [opdict[pauli] for pauli in op]

    pauli_product = paulis[0]
    for p in paulis[1:]:
        pauli_product = pauli_product @ p

    operators += pauli_product

print(operators)

TypeError: unsupported operand type(s) for @: 'ABCMeta' and 'ABCMeta'

In [ ]:
from qiskit.opflow import I, X, Y, Z

opdict = {"I": I, "X": X, "Y": Y, "Z": Z}

# Initialise the operator with the identity contribution
qiskit_op = intermediate["I"*n_qubits] * I.tensorpower(n_qubits)

for op, value in intermediate.items():

    if op == "I"*n_qubits:
        continue

    paulis = [opdict[pauli] for pauli in op]

    pauli_product = paulis[0]
    for p in paulis[1:]:
        pauli_product = pauli_product.tensor(p)

    qiskit_op += value * pauli_product

print(qiskit_op)

In [ ]:
type(qiskit_op)

In [ ]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import VQE
from qiskit import Aer
from qiskit_nature.circuit.library import UCCSD

from qiskit_nature.algorithms import GroundStateEigensolver


In [ ]:
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

q_instance = QuantumInstance(backend=Aer.get_backend('qasm_simulator'))
optimizer = COBYLA(maxiter=80)
esp = ElectronicStructureProblem()



vqe = VQEUCCFactory(q_instance, optimizer).get_solver()


In [ ]:
algorithm_globals.random_seed = 50
ansatz = UCCSD(num_orbitals=n_qubits*2, num_particles=n_act_mos*2)

counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

vqe = VQE(ansatz, optimizer, callback=store_intermediate_result,
            quantum_instance=q_instance)
result = vqe.compute_minimum_eigenvalue(operator=qiskit_op)
print('\rOptimization complete      ')

In [ ]:
pylab.rcParams['figure.figsize'] = (12, 8)
pylab.plot(counts, values, label=type(optimizer).__name__)
pylab.xlabel('Eval count')
pylab.ylabel('Energy')
pylab.title('Energy convergence for various optimizers')
pylab.legend(loc='upper right');

referenece values

In [ ]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=H2_op)
ref_value = result.eigenvalue.real
print(f'Reference value: {ref_value:.5f}')

In [ ]:
pylab.rcParams['figure.figsize'] = (12, 8)
pylab.plot(counts, abs(ref_value - values), label=type(optimizer).__name__)
pylab.xlabel('Eval count')
pylab.ylabel('Energy difference from solution reference value')
pylab.title('Energy convergence for various optimizers')
pylab.yscale('log')
pylab.legend(loc='upper right');